# Linear Support Vector Machine

As we saw in lectures, soft margin SVM uses **hinge loss** $L(y, z) = \max(0, 1-yz)$. This is in contrast to the Perceptron's loss function $L(y,z) = \max(0, -yz)$. In addition, while Perceptron uses SGD with a learning rate of $\alpha=1$, we can choose other procedures.

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
class GradientDescent:
    
    def __init__(self, learning_rate=.001, max_iters=1e4, epsilon=1e-8, record_history=False):
        self.learning_rate = learning_rate
        self.max_iters = max_iters
        self.record_history = record_history
        self.epsilon = epsilon
        if record_history:
            self.w_history = []
            
    def run(self, gradient_fn, x, y, w):
        grad = np.inf
        t = 1
        while np.linalg.norm(grad) > self.epsilon and t < self.max_iters:
            grad = gradient_fn(x, y, w)
            w = w - self.learning_rate * grad
            if self.record_history:
                self.w_history.append(w)
            t += 1
        return w

Below is a simple implementation of Linear SVM, where the only difference with our previous implementation of logistic regression is the choice of loss function and the fact that the input labels are in $\{-1,+1\}$ rather than $\{0,1\}$ (note that in the implementation below, to keep things simple, we are applying the L2 regularization to the intercept as well.)

In [ ]:
def cost_fn(x, y, w, lambdaa):
    N, D = x.shape                                      # not really used!
    z = np.dot(x, w)                                    # N
    J = np.mean(np.maximum(0, 1- y*z)) + (lambdaa/2.) * np.linalg.norm(w)**2  #loss of the SVM
    return J

class LinearSVM:    
    def __init__(self, add_bias=True, lambdaa = .01):
        self.add_bias = add_bias
        self.lambdaa = .01
        pass
            
    def fit(self, x, y, optimizer):
        if x.ndim == 1:
            x = x[:, None]
        if self.add_bias:
            N = x.shape[0]
            x = np.column_stack([x,np.ones(N)])
        N,D = x.shape
        y = 2*y - 1                                     # converting 0,1 to -1,+1
        
        def subgradient(x, y, w):
            N,D = x.shape
            yh = np.dot(x, w)
            violations = np.nonzero(yh*y < 1)[0]                  # get those indexes for which yh*y<1
            grad = -np.dot(x[violations,:].T, y[violations])/N    #compute x^Ty for those indexes and scale it down by N
            grad += self.lambdaa * w                              #add the gradients from the weight regularization term
            return grad

        w0 = np.zeros(D)
        self.w = optimizer.run(subgradient, x, y, w0)
        return self
    
    def predict(self, x):
        if self.add_bias:
            x = np.column_stack([x,np.ones(N)])
        yh = (np.sign(x@self.w) + 1)//2                           #converting -1,+1 to 0,1
        return yh

Let's try again to fit the Iris dataset of previous example this time using linear SVM; this is the setting where the data is not linearly separable.

In [ ]:
dataset = datasets.load_iris()
x, y = dataset['data'][:,[1,2]], dataset['target']
y =  y > 1
optimizer = GradientDescent(learning_rate=.01, max_iters=300, record_history=True)
model = LinearSVM(lambdaa=.00001)
model.fit(x,y, optimizer)
plt.plot(x[y==0,0], x[y==0,1], 'k.' )
plt.plot(x[y==1,0], x[y==1,1], 'b.' )
x_line = np.linspace(np.min(x[:,0]), np.max(x[:,0]), 100)
for t,w in enumerate(optimizer.w_history):
    coef = -w[0]/w[1]
    plt.plot(x_line, coef*x_line - w[2]/w[1], 'r-', alpha=t/len(optimizer.w_history), label=f't={t}')#, alpha=(t+1)/(len(model.w_hist)+.1))
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')
plt.ylim(-1,10)
plt.title('Perceptron when the data is not linearly separable')
plt.show()